50,000 is batch max

In [3]:
import json
import pandas as pd

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

size = 50000
start = 350000
end = start + size

for _ in range(7):
    print(start, end)

    pd.read_parquet('data/chunks_top20_len40_stride40.parquet').iloc[start : end].to_parquet(f'data/chunks_top20_len40_stride40_{start}_{end}.parquet')

    chunks = pd.read_parquet(f'data/chunks_top20_len40_stride40_{start}_{end}.parquet')

    with open(f'data/chunks_top20_len40_stride40_{start}_{end}.jsonl', 'w') as file:
        for row in chunks.to_dict(orient='records'):
            file.write(json.dumps(
                {
                    "custom_id": f"{start}_{end}_{row['chunk_id']}",
                    "method": "POST",
                    "url": "/v1/embeddings",
                    "body": {
                        "model": "text-embedding-3-small",
                        "input": row['chunk'],
                    }
                }
            ) + '\n')

    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

    batch_input_file = client.files.create(
        file=open(f"data/chunks_top20_len40_stride40_{start}_{end}.jsonl", "rb"),
        purpose="batch"
    )

    batch_input_file_id = batch_input_file.id

    client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/embeddings",
        completion_window="24h",
        metadata={
            "description": "nightly eval job"
        }
    )

    start += size
    end += size


350000 400000
400000 450000
450000 500000
500000 550000
550000 600000
600000 650000
650000 700000
